Create store procedure for creating a master date table.

In [ ]:
CREATE OR ALTER PROCEDURE [master].[Initialize_DateTable]
AS
BEGIN
	SET NOCOUNT ON;

	-- Set variables
	DECLARE @StartDate DATE  = '2011-01-01'; -- Start-date
	DECLARE @EndDate DATE =  CAST(YEAR(GETDATE()) + 2 AS CHAR(4)) + '-01-01'; -- End-Date
	   
	DECLARE @DefaultDate DATE = '1900-01-01';
	DECLARE @CurrentDate DATE;

	DECLARE @FiscalStartDate DATE = '2010-09-01';
	DECLARE @FiscalDifference INT = DATEDIFF(m, @FiscalStartDate, @StartDate);

	IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'Holiday' AND XTYPE='U')
	BEGIN
		CREATE TABLE [master].[Holiday](
			[holiday_key] [nchar](18) NOT NULL,
			[actual_date] [datetime] NOT NULL,
			[observed_date] [datetime] NOT NULL,
			[description] [nvarchar](32) NULL,
			[created_date] [datetime] NOT NULL,
			[modified_date] [datetime] NULL,
			CONSTRAINT [PK_master_holiday] PRIMARY KEY CLUSTERED ([holiday_key] ASC),
			INDEX [IX_master_holiday] NONCLUSTERED ([observed_date])
		);
	END

	-- Drop table if it already exists
    IF EXISTS (SELECT * FROM sysobjects WHERE name= N'Date' AND XTYPE='U')
		DROP TABLE [master].[Date];

	-- Create new table
	CREATE TABLE [master].[Date](
		[FullDate] [date] NOT NULL,
		[DayOfWeek] [int] NOT NULL CONSTRAINT DF_Date_DayOfWeek DEFAULT 0,
		[DayOfWeekName] [varchar](10) NOT NULL CONSTRAINT DF_Date_DayOfWeekName DEFAULT '',
		[DayOfWeekAbbrev] [varchar](5) NOT NULL CONSTRAINT DF_Date_DayOfWeekAbbrev DEFAULT '',
		[DayOfMonth] [int] NOT NULL CONSTRAINT DF_Date_DayOfMonth DEFAULT 0,
		[DayOfQuarter] [int] NOT NULL CONSTRAINT DF_Date_DayOfQuarter DEFAULT 0,
		[DayOfYear] [int] NOT NULL CONSTRAINT DF_Date_DayOfYear DEFAULT 0,
		[WeekOfYear] [int] NOT NULL CONSTRAINT DF_Date_WeekOfYear DEFAULT 0,
		[MonthOfYear] [int] NOT NULL CONSTRAINT DF_Date_MonthOfYear DEFAULT 0,
		[MonthName] [varchar](10) NOT NULL CONSTRAINT DF_Date_MonthName DEFAULT '',
		[MonthNameAbbrev] [varchar](5) NOT NULL CONSTRAINT DF_Date_MonthNameAbbrev DEFAULT '',
		[QuarterOfYear] [int] NOT NULL CONSTRAINT DF_Date_QuarterOfYear DEFAULT 0,
		[Year] [int] NOT NULL CONSTRAINT DF_Date_Year DEFAULT 0,
		[FiscalDayOfMonth] [int] NOT NULL CONSTRAINT DF_Date_FiscalDayOfMonth DEFAULT 0,
		[FiscalDayOfQuarter] [int] NOT NULL CONSTRAINT DF_Date_FiscalDayOfQuarter DEFAULT 0,
		[FiscalDayOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalDayOfYear DEFAULT 0,
		[FiscalWeekOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalWeekOfYear DEFAULT 0,
		[FiscalMonthOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalMonthOfYear DEFAULT 0,
		[FiscalQuarterOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalQuarterOfYear DEFAULT 0,
		[FiscalYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalYear DEFAULT 0,
		[IsBusinessDay] [bit] NOT NULL CONSTRAINT DF_Date_IsBusinessDay DEFAULT 0,
		[IsWeekday] [bit] NOT NULL CONSTRAINT DF_Date_IsWeekday DEFAULT 0,
		[IsHoliday] [bit] NOT NULL CONSTRAINT DF_Date_IsHoliday DEFAULT 0,
		[YYYYMM] [char](6) NOT NULL CONSTRAINT DF_Date_YYYYMM DEFAULT '',
		[YYYYMMDD] [char](8) NOT NULL CONSTRAINT DF_Date_YYYYMMDD DEFAULT '',
		[MMMYYYY] [char](9) NOT NULL CONSTRAINT DF_Date_MMMYYYY DEFAULT '',
		[MMYYYY] [char](7) NOT NULL CONSTRAINT DF_Date_MMYYYY DEFAULT '',
		[YYYYQQ] [char](7) NOT NULL CONSTRAINT DF_Date_YYYYQQ DEFAULT '',
		[YYYYFQ] [char](7) NOT NULL CONSTRAINT DF_Date_YYYYFQ DEFAULT '',
		CONSTRAINT [PK_master_date] PRIMARY KEY CLUSTERED ([FullDate] ASC)
	);

	-- Add default date 
	INSERT INTO [master].[Date] ([FullDate], [YYYYMM], [YYYYMMDD], [MMMYYYY], [MMYYYY], [YYYYQQ], [YYYYFQ])
	VALUES (@DefaultDate, '', '', '', '', '', '');

	-- Build new date table data
	INSERT INTO [master].[Date] (
		  [FullDate]
		, [DayOfWeek]
		, [DayOfWeekName]
		, [DayOfWeekAbbrev]
		, [DayOfMonth]
		, [DayOfQuarter]
		, [DayOfYear]
		, [WeekOfYear]
		, [MonthOfYear]
		, [MonthName]
		, [MonthNameAbbrev]
		, [QuarterOfYear]
		, [Year]
		, [FiscalDayOfMonth]
		, [FiscalDayOfQuarter]
		, [FiscalDayOfYear]
		, [FiscalWeekOfYear]
		, [FiscalMonthOfYear]
		, [FiscalQuarterOfYear]
		, [FiscalYear]
		, [IsBusinessDay]
		, [IsWeekday]
		, [IsHoliday]
		, [YYYYMM]
		, [YYYYMMDD]
		, [MMMYYYY]
		, [MMYYYY]
		, [YYYYQQ]
		, [YYYYFQ]
	)
		SELECT
		  a.CurrDate AS [FullDate]
		, DATEPART(DW, a.CurrDate) AS [DayOfWeek]
		, DATENAME(DW, a.CurrDate) AS [DayOfWeekName]
		, CASE WHEN DATEPART(DW, a.CurrDate) IN (3, 5) THEN LEFT(DATENAME(DW, a.CurrDate), 4)
			ELSE LEFT(DATENAME(DW, a.CurrDate), 3)
			END AS [DayOfWeekAbbrev]
		, DATEPART(DD, a.CurrDate) AS [DayOfMonth]
		, DATEDIFF(DD,
			DATEADD(qq, DATEDIFF(qq, 0, a.CurrDate), 0),
			a.CurrDate) + 1
			AS [DayOfQuarter]
		, DATEPART(DY, a.CurrDate) AS [DayOfYear]
		, DATEPART(WW, a.CurrDate) AS [WeekOfYear]
		, DATEPART(MM, a.CurrDate) AS [MonthOfYear]
		, DATENAME(MM, a.CurrDate) AS [MonthName]
		, CASE WHEN DATEPART(MM, a.CurrDate) = 9 THEN LEFT(DATENAME(MM, a.CurrDate), 4)
			ELSE LEFT(DATENAME(MM, a.CurrDate), 3)
			END AS [MonthNameAbbrev]
		, DATEPART(QQ, a.CurrDate) AS [QuarterOfYear]
		, DATEPART(YY, a.CurrDate) AS [Year]
		, DATEPART(DD, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalDayOfMonth]
		, DATEDIFF(DD,
			DATEADD(qq, DATEDIFF(qq, 0, DATEADD(m, @FiscalDifference, a.CurrDate)), 0),
			DATEADD(m, @FiscalDifference, a.CurrDate)) + 1
			AS [FiscalDayOfQuarter]
		, DATEPART(DY, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalDayOfYear]
		, DATEPART(WW, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalWeekOfYear]
		, DATEPART(MM, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalMonthOfYear]
		, DATEPART(QQ, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalQuarterOfYear]
		, DATEPART(YY, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalYear]
		, CASE WHEN DATEPART(DW, a.CurrDate) = 1 OR DATEPART(DW, a.CurrDate) = 7 OR h.[holiday_key] IS NOT NULL THEN 0
			ELSE 1
			END AS [IsBusinessDay]
		, CASE WHEN DATEPART(DW, a.CurrDate) = 1 OR DATEPART(DW, a.CurrDate) = 7 THEN 0
			ELSE 1
			END AS [IsWeekday]
		, CASE WHEN h.[holiday_key] IS NULL THEN 0
			ELSE 1
			END AS [IsHoliday]
		, LEFT(CONVERT(VARCHAR, a.CurrDate, 112), 6) AS [YYYYMM]
		, CONVERT(VARCHAR, a.CurrDate, 112) AS [YYYYMMDD]
		, CASE WHEN DATEPART(MM, a.CurrDate) = 9 THEN LEFT(DATENAME(MM, a.CurrDate), 4) ELSE LEFT(DATENAME(MM, a.CurrDate), 3) END + ' ' + CAST(DATEPART(YY, a.CurrDate) AS char(4)) AS [MMMYYYY]
		, CASE WHEN MONTH(a.CurrDate) < 10 THEN '0' ELSE '' END + CAST(DATEPART(MM, a.CurrDate) AS varchar(4)) + '-' + CAST(DATEPART(YY, a.CurrDate) AS char(4)) AS [MMYYYY]
		, CAST(DATEPART(YY, a.CurrDate) AS char(4)) + '/Q' + CAST(DATEPART(QQ, a.CurrDate) AS char(1)) AS [YYYYQQ]
		, CAST(DATEPART(YY, DATEADD(m, @FiscalDifference, a.CurrDate)) AS char(4)) + '/Q' + CAST(DATEPART(QQ, DATEADD(m, @FiscalDifference, a.CurrDate)) AS char(1)) AS [YYYYFQ]
	FROM (
		SELECT CurrDate = CAST(DATEADD(DAY, rn - 1, @StartDate) AS DATE)
		FROM (
			SELECT TOP (DATEDIFF(DAY, @StartDate, @EndDate)) rn = ROW_NUMBER() OVER (ORDER BY s1.[object_id])
			FROM sys.all_objects AS s1
				CROSS JOIN sys.all_objects AS s2
		    ORDER BY s1.[object_id]) AS x
	) a
		LEFT OUTER JOIN [master].[Holiday] h ON a.CurrDate = h.observed_date;

END
GO

Run the stored procedure to create the new DateTable.

In [ ]:
EXEC [master].[Initialize_DateTable];
GO